# Module Imports

In [43]:
## IMPORT MODULES
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy.interpolate import griddata


In [44]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

# Dataframe

In [45]:
## FULL DATAFRAME
df_raw = pd.read_csv("NL_2-Policy_Summary.csv")

## DROP EXTRANEOUS DATAPOINTS
# df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vz_d']>=2.25)].index)
# df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vx_d']>=0.0)].index)


## GROUP AND AVERAGE RAW DATA BY VEL IC
df_raw_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()




## CLEAN RAW DATA
df = df_raw
df['My_d'] = df['My_d'].apply(lambda x: np.abs(x)) # Convert My to magnitude
# df['My_d'] = df['My_d'].apply(lambda x: 7.7 if x>7.7 else x) # Cap My_d to max moment
df = df.drop(df[(df['vz_d']<= 2.0) & (df['vx_d']<= 0.5)].index) # Drop corner with no successful landings
df = df.dropna()

## AVERAGED DATAFRAME
df_avg = df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

In [46]:
# TEMPLATE FOR LINEAR REGRESSION
from sklearn.linear_model import LinearRegression

def linreg(X ,Y):
    # perform multivariate or univariate
    # X = [var1 var2]
    # y = var3
    reg = LinearRegression(normalize=True)
    reg.fit(X,Y)

    intercept = reg.intercept_ 
    coef = reg.coef_
    
    R2 = reg.score(X,Y) # built in function for r^2
    
    ypred = reg.predict(X) # plot x vs. ypred for line or plane
    
    return ypred, intercept,coef,R2


# Landing Rate Data

In [47]:
## LANDING RATE DATA (RAW)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_raw['vx_d']
Y = df_raw['vz_d']
Z = df_raw['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(df_raw['vx_d'],df_raw['vz_d'],df_raw['landing_rate'])


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)
ax.set_ylim(1.5,4)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Landing Rate (Raw Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_raw_avg['vx_d']
Y = df_raw_avg['vz_d']
Z = df_raw_avg['landing_rate','mean']

# DEFINE COLOR FORMATS AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)

# CREATE PLOTS AND COLOR BAR
ax.plot_trisurf(X,Y,Z,cmap = cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)
ax.set_ylim(1.5,4)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
## AVG LANDING RATE SMOOTHED SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_raw_avg['vx_d']
Y = df_raw_avg['vz_d']
Z = df_raw_avg['landing_rate','mean']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//3))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//3))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
# surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")

# PLOT LIMITS AND INFO
ax.set_zlim(0,1)
ax.set_ylim(1.5,4)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# RREV vs IC

In [50]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df.query(f"My_d<={7.7}")
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data



# LINEAR REGRESSION
X_reg = df_temp[['vx_d','vz_d']]
Y_reg = df_temp['RREV_threshold']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: RREV_reg = {intercept:.2f} + {coef[0]:.2f}*vx_d + {coef[1]:.2f}*vz_d \n")
print(f"R^2 = {R2:.3f}")


# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = df_temp['RREV_threshold']
C = df_temp['RREV_threshold']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=3.5,vmax=5.5)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm,zorder=2)
ax.plot_trisurf(X,Y,ypred,alpha=0.75,label='Linear_Regression',zorder=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")



# PLOT LIMITS AND INFO
ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)


ax.set_xlabel('Vx_d')
ax.set_ylabel('Vz_d')
ax.set_zlabel('RREV_threshold')
ax.set_title('RREV_thr vs IC - (Raw Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: RREV_reg = 5.22 + 0.13*vx_d + -0.28*vz_d 

R^2 = 0.270


In [51]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df.query(f"RREV_threshold>={3.5}")
df_temp_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

# DEFINE VARIABLES
X = df_temp_avg['vx_d']
Y = df_temp_avg['vz_d']
Z = df_temp_avg['RREV_threshold','mean']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//10))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//10))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=3.5,vmax=5.5)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold",orientation="horizontal")

# PLOT LIMITS AND INFO
ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('RREV_threshold')

ax.set_title('Avg RREV_thr vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rotation Time Data

In [52]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['impact_tdelta']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
# # fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs IC (Raw Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# My_d vs IC Data

In [53]:
## Define Dataframe

df_temp = df.query(f"landing_rate>={0.1}")


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

pcm = ax.scatter(df['vx_d'],df['vz_d'],df['My_d'])



ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('My_d [N*mm]')

ax.set_title('My_d vs IC (Raw Data)')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Impact Angle Data

In [54]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df.query(f"impact_eul<={-60}")
df_temp_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

# DEFINE VARIABLES
X = df_temp['vx_d']
Y = df_temp['vz_d']
Z = -df_temp['impact_eul']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle [deg]')

ax.set_title('Impact Angle vs IC (Raw Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df.query(f"impact_eul<={-60}")
df_temp_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

# DEFINE VARIABLES
X = df_temp_avg['vx_d']
Y = df_temp_avg['vz_d']
Z = -df_temp_avg['impact_eul','mean']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//10))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//10))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=60,vmax=180)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Impact Angle [deg]")

# PLOT LIMITS AND INFO
ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle [deg]')

ax.set_title('Avg Impact Angle vs IC')

ax.set_zlim(60,180)
ax.set_ylim(1.5,4.0)
ax.set_zticks([60,90,120,150,180,210])


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Moment vs RREV

In [63]:
fig = plt.figure()
ax = fig.add_subplot(111)

# CLEAN DATA
df_temp = df.query(f"My_d<={50}") 

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.Greys
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV_threshold')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')


ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.1,'Motors Maxed Out \n[My_d = 7.7 N*mm]')




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
df_temp = df.query(f"My_d>={4.0} and RREV_threshold<={3.5}") 
df_temp

,vz_d,vx_d,trial_num,landing_rate,RREV_threshold,G1,RREV_sig,G1_sig,RREV_trigger,OF_y,My_d,impact_eul,impact_tdelta,alpha_mu,alpha_sigma,mu_ini,sigma_ini
152,2.50,1.50,4,1.000000,3.35,7.36,0.00,0.01,3.668000,-2.028000,7.369333,-100.840555,0.406667,[0. 0.],[0. 0.],[1.28 6.56],[1.5 1.5]
226,2.75,2.00,1,1.000000,3.38,7.00,0.00,0.00,3.596000,-2.431000,6.997000,-96.100616,0.450000,[0. 0.],[0. 0.],[2.64 6.1 ],[1.5 1.5]
310,3.00,2.50,3,1.000000,3.15,12.28,0.00,0.41,3.441125,-2.676625,12.217125,-113.363799,0.409500,[0. 0.],[0. 0.],[1.19 6.11],[1.5 1.5]
311,3.00,2.50,4,1.000000,3.15,9.58,0.00,0.02,3.356400,-2.607200,9.563800,-117.026201,0.430000,[0. 0.],[0. 0.],[2.85 6.44],[1.5 1.5]
353,3.25,1.25,5,1.000000,3.11,10.83,0.00,0.00,3.281444,-1.180111,10.833444,-36.955579,0.424000,[0. 0.],[0. 0.],[2.43 6.85],[1.5 1.5]
359,3.25,1.50,5,1.000000,3.22,7.63,0.00,0.05,3.484100,-1.505200,7.653300,-79.706746,0.385200,[0. 0.],[0. 0.],[1.98 6.4 ],[1.5 1.5]
360,3.25,1.75,0,1.000000,3.27,6.52,0.00,0.00,3.457500,-1.741750,6.521125,-101.994689,0.437250,[0. 0.],[0. 0.],[1.9 6.66],[1.5 1.5]
389,3.50,1.00,4,1.000000,2.90,7.27,0.00,0.05,3.201667,-0.870667,7.272667,-91.932349,0.433333,[0. 0.],[0. 0.],[1.05 6.38],[1.5 1.5]
400,3.50,1.50,3,0.500000,3.26,4.12,0.00,0.00,3.697000,-1.483000,4.123000,-90.112999,0.428000,[0. 0.],[0. 0.],[1.41 3.45],[1.5 1.5]
403,3.50,1.75,0,1.000000,3.08,7.81,0.02,0.00,3.269375,-1.530750,7.810875,-74.231769,0.429000,[0. 0.],[0. 0.],[1.39 6.98],[1.5 1.5]


In [57]:
fig = plt.figure()
ax = fig.add_subplot(111)

# CLEAN DATA
df_temp = df.query(f"My_d<={7.7}") 

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['My_d']
C = df_temp['vz_d']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=1.5,vmax=4)
ax.scatter(X,Y,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Vz_d")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV_threshold')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')




ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.1,'Motors Maxed Out \n[My_d = 7.7 N*mm]')




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# POLICY FITTING

## RAW POLICY RELATION

In [58]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df.query(f"My_d<={7.7}")
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data


# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df.query(f"My_d<={7.7}")
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_temp = df_temp.query(f"landing_rate>={0.6}")



# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation (LR >= 60%)')

# Note: the planar look to the data

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# LINEAR MODEL FIT

## Regression with y-based loss

In [60]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df.query(f"My_d<={7.7}")
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg = df_temp.query(f"landing_rate>={0.6}")
df_temp = df_temp.query(f"landing_rate>={0.6}")



# LINEAR REGRESSION
X_reg = df_reg[['RREV_threshold','OF_y']]
Y_reg = df_reg['My_d']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: My_d = {intercept:.2f} + {coef[0]:.2f}*RREV + {coef[1]:.2f}*OF_y \n")
print(f"R^2 = {R2:.3f}")


# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm,zorder=2)
ax.plot_trisurf(X,Y,ypred,alpha=0.75,label='Linear_Regression',zorder=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")


# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation \n(Regression: LR >= 60%)')

# Note: the planar look to the data

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: My_d = -10.44 + 3.49*RREV + 0.47*OF_y 

R^2 = 0.381


In [61]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df.query(f"My_d<={7.7}")
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg = df_temp.query(f"landing_rate>={0.6}")
df_temp = df_temp.query(f"landing_rate>={0.0}")



# LINEAR REGRESSION
X_reg = df_reg[['RREV_threshold','OF_y']]
Y_reg = df_reg['My_d']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: My_d = {intercept:.2f} + {coef[0]:.2f}*RREV + {coef[1]:.2f}*OF_y \n")
print(f"R^2 = {R2:.3f}")


# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm,zorder=2)
ax.plot_trisurf(X_reg['RREV_threshold'],X_reg['OF_y'],ypred,alpha=0.75,label='Linear_Regression',zorder=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")


# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation \n(Regression: LR >= 60%)')

# Note: the planar look to the data

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: My_d = -10.44 + 3.49*RREV + 0.47*OF_y 

R^2 = 0.381


## Regression with x-based loss


In [62]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df.query(f"My_d<={7.7}")
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg = df_temp.query(f"landing_rate>={0.6}")
df_temp = df_temp.query(f"landing_rate>={0.6}")



# LINEAR REGRESSION
X_reg = df_reg[['My_d','OF_y']]
Y_reg = df_reg['RREV_threshold']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: RREV_threshold = {intercept:.2f} + {coef[0]:.2f}*My_d + {coef[1]:.2f}*OF_y \n")
print(f"R^2 = {R2:.3f}")


# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y']
Z = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm,zorder=2)
ax.plot_trisurf(ypred,X_reg['OF_y'],X_reg['My_d'],alpha=0.75,label='Linear_Regression',zorder=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate",orientation='horizontal')


# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation \n(Regression: LR >= 60%)')

# Note: the planar look to the data

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Equation: RREV_threshold = 3.66 + 0.11*My_d + -0.16*OF_y 

R^2 = 0.592
